In [1]:
import requests
import json
import os
import shutil
import pwd
import grp

# Ades2 Deploy and Undeploy Demo

In [2]:
services_folder="/usr/lib/cgi-bin"

## List Processes
Provides a list of all processes

In [3]:
response = requests.get('http://localhost/ogc-api/processes')
response.json()

{'processes': [{'id': 'HelloPy',
   'title': 'Create a welcome message string.',
   'description': 'Create a welcome string.',
   'version': '2.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'http://localhost/ogc-api/processes/HelloPy'},
    {'rel': 'alternate',
     'type': 'text/html',
     'title': 'Process Description',
     'href': 'http://localhost/ogc-api/processes/HelloPy.html'}]},
  {'id': 'UndeployProcess',
   'title': 'Undeploy Process',
   'description': 'This method will undeploy a deployed processing service.',
   'version': '1.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'http://localhost/ogc-api

## Inspect deployed services
In the Ades container/pod inspect the services folder

In [4]:
for path, subdirs, files in os.walk(services_folder):
    for name in files:
        print(os.path.join(path, name))

/usr/lib/cgi-bin/deploy_util.py
/usr/lib/cgi-bin/HelloPy.zcfg
/usr/lib/cgi-bin/UndeployProcess.py
/usr/lib/cgi-bin/DeployProcess.py
/usr/lib/cgi-bin/UndeployProcess.zcfg
/usr/lib/cgi-bin/DeployProcess.zcfg
/usr/lib/cgi-bin/test_service.py
/usr/lib/cgi-bin/main.cfg
/usr/lib/cgi-bin/zoo_loader.cgi
/usr/lib/cgi-bin/oas.cfg


## Deploy the service using the application package
The body of the POST request will include the application package.</br>
The response returns 201 CREATED to indicate that the service has been successfully deployed.</br>
The response Location header provides the path to the service details.

In [5]:
url = "http://localhost/ogc-api/processes/"
payload = open('data/dnbr_app/dnbr.cwl', 'rb').read()

headers = {'Content-Type': 'application/cwl'}
response = requests.request("POST", url, headers=headers, data=payload)

print(f"Body: {response.json()}\n")
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

Body: {'message': 'Service dnbr version 0.1.0 successfully deployed.', 'service': 'dnbr', 'status': 'success'}

Headers: {'Date': 'Tue, 02 Aug 2022 09:39:00 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Content-Length': '104', 'Location': 'http://localhost/ogc-api/processes/dnbr', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json; charset=UTF-8'}

Status code: 201


## Inspect the application service's files

In [6]:
for path, subdirs, files in os.walk("/usr/lib/cgi-bin/"):
    for name in files:
        if "dnbr" in name or "dnbr" in path:
            print(os.path.join(path, name))

/usr/lib/cgi-bin/dnbr.zcfg
/usr/lib/cgi-bin/dnbr/calrissian_runner.py
/usr/lib/cgi-bin/dnbr/__init__.py
/usr/lib/cgi-bin/dnbr/service.py
/usr/lib/cgi-bin/dnbr/app-package.cwl
/usr/lib/cgi-bin/dnbr/test.py


## Execute the deployed service
Request body is json as defined by API Processes to define the inputs and outputs, consistent with the CWL Workflow application package.</br>
The response returns 201 CREATED to indicate that the job has been successfully initiated.</br>
The response Location header provides the path (/processes/{application_name}/jobs/{job_id}) to follow the job status.

In [7]:
url = "http://localhost/ogc-api/processes/dnbr"

payload = open('data/dnbr_app/dnbr_execute_payload.json', 'rb').read()
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

Headers: {'Date': 'Tue, 02 Aug 2022 09:43:51 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Location': 'http://localhost/ogc-api/jobs/9dbc6228-1247-11ed-b735-0242ac192202', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json;charset=UTF-8'}

Status code: 201


In [8]:
location = response.headers["Location"]
print(location)

http://localhost/ogc-api/jobs/9dbc6228-1247-11ed-b735-0242ac192202


## Get Job Status
Check the status of a previously submitted job - using the URL returned in the Location header of the execute request.</br>
The response body json provides a status string (success/running/failed) and a % progress..</br>
In the case of a failure then a descriptive message is provided.

In [9]:
get_status_response = requests.get(location)
get_status_response.json()

{'jobID': '9dbc6228-1247-11ed-b735-0242ac192202',
 'status': 'successful',
 'message': 'ZOO-Kernel successfully run your service!',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'http://localhost/ogc-api/jobs/9dbc6228-1247-11ed-b735-0242ac192202'},
  {'title': 'Result location',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
   'type': 'application/json',
   'href': 'http://localhost/ogc-api/jobs/9dbc6228-1247-11ed-b735-0242ac192202/results'}]}

## Get Job Result
Returns details of the outputs for a successful job execution.</br>
The response body provides json data that includes the reference to the STAC file that indexes the processing outputs.

In [10]:
get_result_url = f"{location}/results"
get_result_response = requests.request("GET", get_result_url)
get_result_response_json=json.loads(get_result_response.text)
print(json.dumps(get_result_response_json,indent=2))

{
  "stac": {
    "href": "http://localhost/temp//ZOO_DATA_dnbr_stac_9dbc6228-1247-11ed-b735-0242ac192202_0.json"
  }
}


## Inspect Job Result

In [11]:
result_url = get_result_response_json["stac"]["href"]
print(result_url)
result_response = requests.request("GET", result_url)
print(result_response.text)

http://localhost/temp//ZOO_DATA_dnbr_stac_9dbc6228-1247-11ed-b735-0242ac192202_0.json
Success!


## Undeploy the service
Undeploys the service

In [13]:
url = "http://localhost/ogc-api/processes/dnbr"
response = requests.request("DELETE", url)
print(response.headers)
print(response.status_code)

{'Date': 'Tue, 02 Aug 2022 09:46:17 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive'}
204


In [14]:
for path, subdirs, files in os.walk(services_folder):
    for name in files:
        print(os.path.join(path, name))

/usr/lib/cgi-bin/deploy_util.py
/usr/lib/cgi-bin/HelloPy.zcfg
/usr/lib/cgi-bin/UndeployProcess.py
/usr/lib/cgi-bin/DeployProcess.py
/usr/lib/cgi-bin/UndeployProcess.zcfg
/usr/lib/cgi-bin/DeployProcess.zcfg
/usr/lib/cgi-bin/test_service.py
/usr/lib/cgi-bin/main.cfg
/usr/lib/cgi-bin/zoo_loader.cgi
/usr/lib/cgi-bin/oas.cfg
/usr/lib/cgi-bin/__pycache__/UndeployProcess.cpython-38.pyc
/usr/lib/cgi-bin/__pycache__/DeployProcess.cpython-38.pyc
/usr/lib/cgi-bin/__pycache__/deploy_util.cpython-38.pyc


# Deploying on namespace

## Creating the namespace
Creating the folder for the namespace

In [15]:
default_services_path="/usr/lib/cgi-bin"
my_namespace_path="/opt/zooservices_namespaces/eoepca"

In [16]:
if not os.path.exists(my_namespace_path):
    os.makedirs(my_namespace_path) 

Copying the Deploy and Undeploy services in the namespace folder

In [17]:
shutil.copyfile(f"{default_services_path}/DeployProcess.py", f"{my_namespace_path}/DeployProcess.py")
shutil.copyfile(f"{default_services_path}/DeployProcess.zcfg", f"{my_namespace_path}/DeployProcess.zcfg")
shutil.copyfile(f"{default_services_path}/UndeployProcess.py", f"{my_namespace_path}/UndeployProcess.py")
shutil.copyfile(f"{default_services_path}/UndeployProcess.zcfg", f"{my_namespace_path}/UndeployProcess.zcfg")

'/opt/zooservices_namespaces/eoepca/UndeployProcess.zcfg'

Granting permissions to the namespace folder

In [18]:
user_id=pwd.getpwnam("www-data")[2]
group_id=grp.getgrnam('www-data')[2]

In [19]:
for dirpath, dirnames, filenames in os.walk(my_namespace_path):
    shutil.chown(dirpath, user_id, group_id)
    for filename in filenames:
        shutil.chown(os.path.join(dirpath, filename), user_id,group_id)

## Inspect namespace services
Provides a list of all services in the namespace *eoepca*

In [20]:
response = requests.get('http://localhost/eoepca/ogc-api/processes')
response.json()

{'processes': [{'id': 'UndeployProcess',
   'title': 'Undeploy Process',
   'description': 'This method will undeploy a deployed processing service.',
   'version': '1.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'http://localhost/eoepca/ogc-api/processes/UndeployProcess'},
    {'rel': 'alternate',
     'type': 'text/html',
     'title': 'Process Description',
     'href': 'http://localhost/eoepca/ogc-api/processes/UndeployProcess.html'}]},
  {'id': 'DeployProcess',
   'title': 'Deploys a zoo service from an application package.',
   'description': 'Deploys a zoo service from an application package.',
   'version': '2.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/js

## Deploying application package

In [22]:
url = "http://localhost/eoepca/ogc-api/processes/"
payload = open('data/dnbr_app/dnbr.cwl', 'rb').read()

headers = {'Content-Type': 'application/cwl'}
response = requests.request("POST", url, headers=headers, data=payload)

print(f"Body: {response.json()}\n")
print(f"Headers: {response.headers}")
print(f"Status code: {response.status_code}")

Body: {'message': 'Service dnbr version 0.1.0 successfully deployed.', 'service': 'dnbr', 'status': 'success'}

Headers: {'Date': 'Tue, 02 Aug 2022 09:49:22 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Content-Length': '104', 'Location': 'http://localhost/eoepca/ogc-api/processes/dnbr', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json; charset=UTF-8'}
Status code: 201


In [23]:
response = requests.get('http://localhost/eoepca/ogc-api/processes/dnbr')
response.json()

{'id': 'dnbr',
 'title': 'dNBR - produce the delta normalized difference between NIR and SWIR 22 over a pair of stac items',
 'description': 'dNBR - produce the delta normalized difference between NIR and SWIR 22 over a pair of stac items',
 'version': '0.1.0',
 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
 'outputTransmission': ['value', 'reference'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'http://localhost/eoepca/ogc-api/processes/dnbr/execution'},
  {'rel': 'alternate',
   'type': 'text/html',
   'title': 'Execute End Point',
   'href': 'http://localhost/eoepca/ogc-api/processes/dnbr/execution.html'}],
 'inputs': {'pre_stac_item': {'title': 'pre_stac_item',
   'description': 'Pre-event Sentinel-2 item',
   'schema': {'type': 'string', 'default': 'Any value'}},
  'post_stac_item': {'title': 'post_stac_item',
   'description': 'Post-event Sentinel-2 item',
   

## Execute the application

In [24]:
url = "http://localhost/eoepca/ogc-api/processes/dnbr"

payload = open('data/dnbr_app/dnbr_execute_payload.json', 'rb').read()
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

Headers: {'Date': 'Tue, 02 Aug 2022 09:49:40 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Location': 'http://localhost/eoepca/ogc-api/jobs/6d8d5692-1248-11ed-ab44-0242ac192202', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json;charset=UTF-8'}

Status code: 201


In [25]:
location = response.headers["Location"]
print(location)

http://localhost/eoepca/ogc-api/jobs/6d8d5692-1248-11ed-ab44-0242ac192202


## Get Job Status

In [26]:
get_status_response = requests.get(location)
get_status_response.json()

{'jobID': '6d8d5692-1248-11ed-ab44-0242ac192202',
 'status': 'successful',
 'message': 'ZOO-Kernel successfully run your service!',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'http://localhost/eoepca/ogc-api/jobs/6d8d5692-1248-11ed-ab44-0242ac192202'},
  {'title': 'Result location',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
   'type': 'application/json',
   'href': 'http://localhost/eoepca/ogc-api/jobs/6d8d5692-1248-11ed-ab44-0242ac192202/results'}]}

## Get Job Result

In [27]:
get_result_url = f"{location}/results"
get_result_response = requests.request("GET", get_result_url)
get_result_response_json=json.loads(get_result_response.text)
print(json.dumps(get_result_response_json,indent=2))

{
  "stac": {
    "href": "http://localhost/temp//ZOO_DATA_dnbr_stac_6d8d5692-1248-11ed-ab44-0242ac192202_0.json"
  }
}


## Inspect Job Result

In [28]:
result_url = get_result_response_json["stac"]["href"]
print(result_url)
result_response = requests.request("GET", result_url)
print(result_response.text)

http://localhost/temp//ZOO_DATA_dnbr_stac_6d8d5692-1248-11ed-ab44-0242ac192202_0.json
Success!


## Undeploy the service

In [30]:
url = "http://localhost/eoepca/ogc-api/processes/dnbr"
response = requests.request("DELETE", url)
print(response.headers)
print(response.status_code)

{'Date': 'Tue, 02 Aug 2022 09:50:24 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive'}
204


In [31]:
for path, subdirs, files in os.walk(my_namespace_path):
    for name in files:
        print(os.path.join(path, name))

/opt/zooservices_namespaces/eoepca/UndeployProcess.py
/opt/zooservices_namespaces/eoepca/DeployProcess.py
/opt/zooservices_namespaces/eoepca/UndeployProcess.zcfg
/opt/zooservices_namespaces/eoepca/DeployProcess.zcfg
/opt/zooservices_namespaces/eoepca/__pycache__/UndeployProcess.cpython-38.pyc
/opt/zooservices_namespaces/eoepca/__pycache__/DeployProcess.cpython-38.pyc
